Aqui importamos todas as ferramentas que serão utilizadas: visualização, manipulação de dados, algoritmos de Machine Learning, avaliação de desempenho, interface gráfica e salvamento de modelos

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo

c:\Users\Pichau\Desktop\Projetos\IA_Formacao.4.0\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Carregando o Dataset que vai ser usado

In [2]:
dataset = fetch_ucirepo(id=17)  
X_full = dataset.data.features
y = dataset.data.targets.iloc[:, 0]  

Aqui selecionamos apenas 10 variáveis do dataset para seguir a regra do projeto

In [3]:
features_escolhidas = [
    "radius1", "texture1", "perimeter1", "area1",
    "smoothness1", "radius2", "texture2", "perimeter2",
    "area2", "smoothness2"
]
X = X_full[features_escolhidas]

Verificamos se o dataset está balanceado entre os casos benignos e malignos

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Essa parte executa o processo de treino/teste 30 vezes. Isso permite verificar a média de acertos e a estabilidade (desvio padrão) dos modelos

In [5]:
accs_lr = []
accs_dt = []

for _ in range(30):
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, stratify=y, random_state=None
    )

    modelo_lr = LogisticRegression(max_iter=1000)
    modelo_dt = DecisionTreeClassifier()

    modelo_lr.fit(X_train, y_train)
    modelo_dt.fit(X_train, y_train)

    pred_lr = modelo_lr.predict(X_test)
    pred_dt = modelo_dt.predict(X_test)

    accs_lr.append(accuracy_score(y_test, pred_lr))
    accs_dt.append(accuracy_score(y_test, pred_dt))

Comparamos qual modelo teve maior acurácia média (melhor desempenho) e qual teve menor desvio padrão (mais estável)

In [6]:
media_lr = np.mean(accs_lr)
std_lr = np.std(accs_lr)
media_dt = np.mean(accs_dt)
std_dt = np.std(accs_dt)

print("\n Resultados após 30 execuções:")
print(f"Regressão Logística - Média: {media_lr:.4f} | Desvio Padrão: {std_lr:.4f}")
print(f"Árvore de Decisão    - Média: {media_dt:.4f} | Desvio Padrão: {std_dt:.4f}")


 Resultados após 30 execuções:
Regressão Logística - Média: 0.9361 | Desvio Padrão: 0.0132
Árvore de Decisão    - Média: 0.8871 | Desvio Padrão: 0.0232


Escolhemos o modelo com maior média de acurácia

In [7]:
if media_lr > media_dt:
    melhor_modelo = modelo_lr
    modelo_nome = "Regressão Logística"
else:
    melhor_modelo = modelo_dt
    modelo_nome = "Árvore de Decisão"

print(f"\n Modelo selecionado para salvar: {modelo_nome}")


 Modelo selecionado para salvar: Regressão Logística


Aqui salvamos o modelo com melhor desempenho na biblioteca pickle

In [8]:
with open("modelo_salvo.pkl", "wb") as f:
    pickle.dump(melhor_modelo, f)

with open("scaler_salvo.pkl", "wb") as f:
    pickle.dump(scaler, f)

Essa interface simples com Gradio permite que o usuário preencha os valores manualmente e receba uma predição amigável sobre o tumor

In [9]:
def prever_tumor(*inputs):
    with open("modelo_salvo.pkl", "rb") as f:
        modelo = pickle.load(f)
    with open("scaler_salvo.pkl", "rb") as f:
        scaler = pickle.load(f)
    dados = np.array(inputs).reshape(1, -1)
    dados_escalados = scaler.transform(dados)
    pred = modelo.predict(dados_escalados)[0]
    return " Benigno" if pred == 0 else " Maligno"

inputs = [gr.Number(label=nome) for nome in features_escolhidas]
gr.Interface(fn=prever_tumor, inputs=inputs, outputs="text", title="Classificador de Tumor Mamário").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
